# Main Feedback Generation Notebook
This notebook handles preprocessing, model interaction, and feedback generation.

**--Set up: Github, Paths, Imports, Installs**

In [9]:
# Mount Google Drive (optional, you'll get a prompt to authorize account)
# from google.colab import drive
# drive.mount('/content/drive')

# Start in root Colab directory to avoid nesting
%cd /content

# Clone your GitHub repo (replace with your actual repo URL)
!git clone https://github.com/ML-name/project.git
%cd project

# List all branches (optional, for checking)
# !git branch -a

# Checkout YOUR branch (!!replace "your-branch-name"!!)
!git checkout -b model/prompting origin/model/prompting

%pip install -r requirements.txt --quiet
!pip install --upgrade openai --quiet

/content
fatal: destination path 'project' already exists and is not an empty directory.
/content/project
fatal: A branch named 'model/prompting' already exists.


**--Default imports**

In [10]:
import os
from tropos import StudentSubmission

# Add src folder to python path to edit python files
import sys
sys.path.append('/content/project/')

from google.colab import userdata
import os
openai_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = openai_key


**--Import modules (youre working on)**
<br>*each of our classes will be what will merge to this notebook (im p sure)*
<br>only loads what you explicitly request
<br>(this helps keep memory low and import fast)
<br> *the following is an example with my Rubric module*

In [15]:
from tropos import test_feedback_console

test_feedback_console(
    requirements_path="/content/Requirements.docx",
    example_dir="/content/project/data/raw",
    target_dir="/content/project/data/unmarked_raw",
    output_dir="/content/project/data/generated_output",
    verbose=True
)


ImportError: cannot import name 'test_feedback_console' from 'tropos' (/content/project/tropos/__init__.py)